In [ ]:
import pygame
import numpy as np
from pygame.locals import *

# Rotaciona os eixos
rotate_x=np.array([
    [1, 0, 0],
    [0, 0, -1],
    [0, 1, 0]
])

rotate_y=np.array([
    [0,  0, -1],
    [0,  1,  0],
    [1,  0,  0]
])

rotate_z=np.array([
    [0, -1, 0],
    [1,  0, 0],
    [0,  0, 1]
])

class CuboMagico:
    def __init__(self):
        self.vertices = [np.array([x, y, z]) for x in (-1, 1) for y in (-1, 1) for z in (-1, 1)]
        self.arestas = [
            (0, 1), (1, 3), (3, 2), (2, 0),
            (4, 5), (5, 7), (7, 6), (6, 4),
            (0, 4), (1, 5), (2, 6), (3, 7)
        ]
        self.angulos = np.array([0, 0, 0])
        self.centro = np.array([400, 300])

    def rotacionar(self, eixo, angulo):
        matriz = np.eye(3)
        if eixo == 'x':
            matriz = rotate_x
        elif eixo == 'y':
            matriz = rotate_y
        elif eixo == 'z':
            matriz = rotate_z

        rotacao = np.linalg.matrix_power(matriz, int(angulo))
        self.vertices = [np.dot(v, rotacao) for v in self.vertices]

    def projetar(self, v):
        fator_projecao = 400 / (v[2] + 5)
        x = int(v[0] * fator_projecao) + self.centro[0]
        y = int(v[1] * fator_projecao) + self.centro[1]
        return (x, y)

    def desenhar(self, tela):
        for aresta in self.arestas:
            pontos = [self.projetar(self.vertices[i]) for i in aresta]
            pygame.draw.line(tela, (255, 255, 255), pontos[0], pontos[1], 2)

def main():
    pygame.init()
    tela = pygame.display.set_mode((800, 600))
    pygame.display.set_caption("Cubo Mágico")

    cubo = CuboMagico()
    clock = pygame.time.Clock()

    rotando_x, rotando_y = False, False
    angulo_x, angulo_y = 0, 0

    rodando = True
    while rodando:
        tela.fill((0, 0, 0))
        
        for evento in pygame.event.get():
            if evento.type == QUIT:
                rodando = False
            elif evento.type == KEYDOWN:
                if evento.key == K_x:
                    rotando_x = True
                elif evento.key == K_y:
                    rotando_y = True
            elif evento.type == KEYUP:
                if evento.key == K_x:
                    rotando_x = False
                elif evento.key == K_y:
                    rotando_y = False

        # Rotação por tecla
        if rotando_x:
            cubo.rotacionar('x', 1)
        if rotando_y:
            cubo.rotacionar('y', 1)

        # Desenha cubo
        cubo.desenhar(tela)

        pygame.display.flip()
        clock.tick(30)

    pygame.quit()

if __name__ == "__main__":
    main()


In [2]:
import pygame
import math
import numpy as np

# Configurações iniciais
pygame.init()
screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Cubo Mágico 3x3x3")

# Função para projetar um ponto 3D para 2D
def project_3d_to_2d(point, fov, viewer_distance):
    factor = fov / (viewer_distance + point[2])
    x = point[0] * factor + 400  # Centraliza na tela (800px de largura)
    y = -point[1] * factor + 300  # Centraliza na tela (600px de altura)
    return (x, y)

# Rotação de matriz
def rotate(point, angle_x, angle_y):
    # Rotação no eixo X
    cos_x = math.cos(angle_x)
    sin_x = math.sin(angle_x)
    y = point[1] * cos_x - point[2] * sin_x
    z = point[1] * sin_x + point[2] * cos_x
    point[1], point[2] = y, z

    # Rotação no eixo Y
    cos_y = math.cos(angle_y)
    sin_y = math.sin(angle_y)
    x = point[0] * cos_y - point[2] * sin_y
    z = point[0] * sin_y + point[2] * cos_y
    point[0], point[2] = x, z

    return point

# Função para criar os vértices de um cubo 3D
def create_cube(size, offset):
    half = size / 2
    vertices = [
        [-half, half, -half],
        [half, half, -half],
        [half, -half, -half],
        [-half, -half, -half],
        [-half, half, half],
        [half, half, half],
        [half, -half, half],
        [-half, -half, half]
    ]
    # Ajusta para a posição de offset (colocando em um grid 3x3x3)
    for v in vertices:
        v[0] += offset[0]
        v[1] += offset[1]
        v[2] += offset[2]
    return vertices

# Definição das faces (usando índices dos vértices)
faces = [
    [0, 1, 2, 3],  # Face de trás
    [4, 5, 6, 7],  # Face da frente
    [0, 1, 5, 4],  # Face de cima
    [2, 3, 7, 6],  # Face de baixo
    [0, 3, 7, 4],  # Face da esquerda
    [1, 2, 6, 5]   # Face da direita
]

# Parâmetros de visão
fov = 256
viewer_distance = 6
angle_x, angle_y = 0, 0

# Controladores de movimento
mouse_drag = False
last_mouse_pos = (0, 0)

# Tamanho do cubo menor (1x1x1 cubo de Rubik)
cube_size = 1
# Cria cubos menores dentro de um grid 3x3x3
cubes = []
for x in range(-1, 2):
    for y in range(-1, 2):
        for z in range(-1, 2):
            cubes.append(create_cube(cube_size, (x * cube_size * 1.1, y * cube_size * 1.1, z * cube_size * 1.1)))

# Loop principal
running = True
while running:
    screen.fill((0, 0, 0))  # Limpa a tela

    # Verifica eventos
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:  # Clique esquerdo para arrastar
                mouse_drag = True
                last_mouse_pos = pygame.mouse.get_pos()
        elif event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1:  # Parar de arrastar
                mouse_drag = False
        elif event.type == pygame.MOUSEMOTION:
            if mouse_drag:
                mouse_x, mouse_y = pygame.mouse.get_pos()
                dx = mouse_x - last_mouse_pos[0]
                dy = mouse_y - last_mouse_pos[1]
                angle_x += dy * 0.01  # Controla a velocidade de rotação
                angle_y += dx * 0.01
                last_mouse_pos = (mouse_x, mouse_y)

    # Para cada cubo menor
    for cube in cubes:
        transformed_vertices = []
        for vertex in cube:
            rotated_vertex = rotate(vertex[:], angle_x, angle_y)
            projected_vertex = project_3d_to_2d(rotated_vertex, fov, viewer_distance)
            transformed_vertices.append(projected_vertex)

        # Desenha as faces do cubo menor
        for face in faces:
            point_list = [transformed_vertices[i] for i in face]
            pygame.draw.polygon(screen, (255, 255, 255), point_list, 1)

    # Atualiza a tela
    pygame.display.flip()
    pygame.time.Clock().tick(60)

pygame.quit()
